In [1]:
from mlclasses_f3dasm import LinearRegression, MLArchitecture, get_reshaped_array_from_list_of_arrays, get_flat_array_from_list_of_arrays
import tensorflow as tf
import f3dasm
import numpy as np
from typing import List
from copy import copy
import pandas as pd
from sklearn.model_selection import train_test_split

2022-12-21 12:27:12.260803: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 12:27:12.479957: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-21 12:27:13.521916: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64:/usr/local/cuda-11.1/lib64
2022-12-21 12:27:13.522064: W ten

In [2]:
from d2l import tensorflow as d2l

In [3]:
class DataModule:
    """Defined in :numref:`subsec_oo-design-models`"""
    def __init__(self, root='../data'):
        self.root= root

    def get_dataloader(self, train):
        raise NotImplementedError

    def train_dataloader(self):
        return self.get_dataloader(train=True)

    def val_dataloader(self):
        return self.get_dataloader(train=False)

    def get_tensorloader(self, tensors, train, indices=slice(0, None)):
        """Defined in :numref:`sec_synthetic-regression-data`"""
        tensors = tuple(a[indices] for a in tensors)
        shuffle_buffer = tensors[0].shape[0] if train else 1
        return tf.data.Dataset.from_tensor_slices(tensors).shuffle(
            buffer_size=shuffle_buffer).batch(self.batch_size)

In [4]:
num_train = 1000
num_val = 1000
n = num_train + num_val
b = 4.2
noise_multiplier = 0.01
w = tf.constant([2, -3.4])
noise = tf.random.normal((n, 1)) * noise_multiplier

X = tf.random.normal((n, w.shape[0])) # (num, dim)
y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b + noise # (1, dim)

2022-12-21 12:27:25.451751: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-21 12:27:25.451852: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martin): /proc/driver/nvidia/version does not exist
2022-12-21 12:27:25.453667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
1001 // 2


500

In [6]:
i = tf.constant([0. , 1.])
len(i)

2

In [7]:
def create_regression_data(n: int, b: float, w: List[float], noise_multiplier: float = 0.01) -> f3dasm.Data:
    w = tf.constant(w)
    dim = len(w)
    num_train = n // 2
    num_val = n - num_train

    noise = tf.random.normal((n, 1)) * noise_multiplier

    X = tf.random.normal((n, w.shape[0])) # (num, dim)
    y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b + noise # (1, dim)

    # Create designspace
    design = f3dasm.make_nd_continuous_design(bounds=np.tile([-np.inf,np.inf], (dim, 1)), dimensionality=dim)

    # Create Data object
    regression_data = f3dasm.Data(design)

    regression_data.add_numpy_arrays(input=X.numpy(), output=y.numpy())
    return regression_data

In [8]:
reg_data = create_regression_data(n=2000, b=4.2, w=[2, -3.4, 5., 6.7, -3.4])
reg_data.data

input                                             output
            x0        x1        x2        x3        x4          y
0     0.232323  1.372074  0.239957 -0.152301 -0.373612   1.439567
1    -0.270130 -0.729988  0.677494  0.310054 -3.026062  21.920729
2    -1.653078 -0.652458 -1.817044  0.496654 -0.959227   0.606139
3     0.797568 -0.295403 -0.213270 -0.704185 -0.032756   1.123363
4     1.062434 -0.363580 -1.267274 -1.586046  0.017212  -9.461775
...        ...       ...       ...       ...       ...        ...
1995 -0.465056 -0.326835  1.198465 -1.215369  0.895153  -0.813348
1996 -0.022622 -2.979682  1.005019 -0.926413  0.347308  11.925545
1997 -1.001367 -0.821210 -0.917331 -0.062754  0.305457  -1.046961
1998 -0.694200 -2.073502  0.880892 -0.912830 -0.003548   8.165611
1999 -1.032161  1.047530  1.636716  0.802599  1.004212   8.718799

[2000 rows x 6 columns]

In [9]:
train, test = reg_data.split_to_train_test(test_size=0.2)
train

input                                             output
            x0        x1        x2        x3        x4          y
1709 -0.460329 -2.022260 -0.385206 -0.756874 -1.213675   7.287460
689  -1.172031 -0.561345  2.406005 -1.833227  0.610663   1.428991
22    0.910888 -0.819529 -0.884486 -0.374983 -0.303449   2.897491
234   0.128376  1.273280 -0.487071  0.157668  0.470714  -2.854374
579  -0.440045  0.356057 -0.354154  0.340894 -0.906325   5.713332
...        ...       ...       ...       ...       ...        ...
1676 -0.584667  0.833516 -1.151453  0.734843  0.573349  -2.602968
129   0.609697  1.524423  1.152034  0.195733 -0.133834   7.758787
900   2.200780  0.421673 -1.870648  0.146652 -0.682617   1.116112
826  -0.226725 -1.051592  0.505996 -0.064839 -1.579572  14.801998
1600  1.363118  0.508119  0.710688  1.283337 -0.607225  19.425373

[1600 rows x 6 columns]

In [10]:
batches = reg_data.split_to_batches(data=train, number_of_batches=7)
batches

[         input                                             output
             x0        x1        x2        x3        x4          y
 1709 -0.460329 -2.022260 -0.385206 -0.756874 -1.213675   7.287460
 689  -1.172031 -0.561345  2.406005 -1.833227  0.610663   1.428991
 22    0.910888 -0.819529 -0.884486 -0.374983 -0.303449   2.897491
 234   0.128376  1.273280 -0.487071  0.157668  0.470714  -2.854374
 579  -0.440045  0.356057 -0.354154  0.340894 -0.906325   5.713332
 ...        ...       ...       ...       ...       ...        ...
 1030  0.332746 -0.841317 -0.353050 -0.359378  0.474521   1.928897
 1098  0.654703  1.688435 -0.187183  0.736782 -1.366534   8.422804
 1874  0.640359  0.486049 -0.541170 -0.940912  0.000187  -5.179186
 1967 -1.550178  0.377595  0.966798  0.497803 -0.264029   8.882068
 1317  0.321732 -1.817388  0.696517 -0.937991 -0.851060  11.123124
 
 [229 rows x 6 columns],
          input                                             output
             x0        x1        x2

In [38]:
batches[0]['input'].to_numpy()

array([[-4.60329413e-01, -2.02226019e+00, -3.85206282e-01,
        -7.56873608e-01, -1.21367514e+00],
       [-1.17203116e+00, -5.61344862e-01,  2.40600514e+00,
        -1.83322656e+00,  6.10662937e-01],
       [ 9.10888374e-01, -8.19529295e-01, -8.84485841e-01,
        -3.74983251e-01, -3.03448737e-01],
       ...,
       [ 6.40358746e-01,  4.86048996e-01, -5.41169584e-01,
        -9.40911829e-01,  1.87421771e-04],
       [-1.55017805e+00,  3.77594799e-01,  9.66798365e-01,
         4.97803122e-01, -2.64029145e-01],
       [ 3.21731776e-01, -1.81738830e+00,  6.96517348e-01,
        -9.37991381e-01, -8.51059794e-01]])

In [44]:
m = LinearRegression(dimensionality=5)
m.loss(m(batches[0]['input'].to_numpy()),batches[0]['output'].to_numpy())

<tf.Tensor: shape=(), dtype=float32, numpy=114.73258>

In [43]:
with tf.GradientTape() as tape:
    loss = m.loss(m(batches[0]['input'].to_numpy()), batches[0]['output'].to_numpy())

grads = tape.gradient(loss, m.trainable_variables)
loss, grads

(<tf.Tensor: shape=(), dtype=float32, numpy=114.69507>,
 [<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
  array([[ -3.1385741],
         [  5.6349955],
         [ -8.081214 ],
         [-15.644523 ],
         [  3.6313984]], dtype=float32)>,
  <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-9.69441], dtype=float32)>])

In [11]:
X.numpy()

array([[-0.21225202, -0.01655633],
       [-1.7237953 , -0.19700338],
       [-0.14077929, -1.2567658 ],
       ...,
       [-0.4010439 , -0.6913797 ],
       [ 0.23575255, -2.1456401 ],
       [ 1.6914388 , -0.15264705]], dtype=float32)

In [12]:
y.numpy()

array([[ 3.830437 ],
       [ 1.4253321],
       [ 8.188075 ],
       ...,
       [ 5.7421474],
       [11.965922 ],
       [ 8.104585 ]], dtype=float32)

In [13]:
class SyntheticRegressionData(DataModule):
    """Defined in :numref:`sec_synthetic-regression-data`"""
    def __init__(self, w, b, noise_value=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        self.w = w
        self.b = b
        self.noise_value = noise_value
        self.num_train = num_train
        self.num_val = num_val
        self.batch_size = batch_size

        self.X, self.y = self._create_data()

    def _create_data(self):
        n = self.num_train + self.num_val
        noise = tf.random.normal((n, 1)) * self.noise_value
        X = tf.random.normal((n, self.w.shape[0]))
        y = tf.matmul(X, tf.reshape(self.w, (-1, 1))) + self.b + noise
        return X, y

    def get_dataloader(self, train):
        """Defined in :numref:`sec_synthetic-regression-data`"""
        i = slice(0, self.num_train) if train else slice(self.num_train, None)
        return self.get_tensorloader((self.X, self.y), train, i)

In [14]:
class Trainer:
    """Defined in :numref:`subsec_oo-design-models`"""
    def __init__(self, max_epochs, gradient_clip_val=0):
        self.max_epochs = max_epochs
        self.gradient_clip_val = gradient_clip_val

        # self.optim = tf.keras.optimizers.SGD(0.03)

    def prepare_data(self, data: DataModule):
        self.train_dataloader = data.train_dataloader()
        self.val_dataloader = data.val_dataloader()
        self.num_train_batches = len(self.train_dataloader)
        self.num_val_batches = (len(self.val_dataloader)
                                if self.val_dataloader is not None else 0)

    def prepare_model(self, model: MLArchitecture):
        self.model = model

    def prepare_optimizer(self, optimizer):
        self.optimizer = optimizer

    def fit(self, model: MLArchitecture, data: DataModule, optimizer):
        self.prepare_data(data)
        self.prepare_model(model)
        self.prepare_optimizer(optimizer)
        # self.optim = model.configure_optimizers()
        self.epoch = 0
        self.train_batch_idx = 0
        self.val_batch_idx = 0
        for self.epoch in range(self.max_epochs):
            self.fit_epoch()

    def extract_model(self):
        return self.model

    def prepare_batch(self, batch):
        """Defined in :numref:`sec_linear_scratch`"""
        return batch

    def evaluate(self, X, Y_true):
        with tf.GradientTape() as tape:
            loss = self.model.loss(self.model(X), Y_true)

        grads = tape.gradient(loss, self.model.trainable_variables)
        return loss, grads



    def fit_epoch(self):
        """Defined in :numref:`sec_linear_scratch`"""
        # self.model.training = True
        for batch in self.train_dataloader:
            
            loss, grads = self.evaluate(*batch[:-1],batch[-1])

            # Optimization update
            w = self.model.get_model_weights()
            update = w - (0.03 * get_flat_array_from_list_of_arrays(grads))
            self.model.set_model_weights(update)

            self.train_batch_idx += 1


        if self.val_dataloader is None:
            return

            
        # self.model.training = False
        for batch in self.val_dataloader:
            self.model.validation_step(self.prepare_batch(batch))
            self.val_batch_idx += 1

In [15]:
dim = 3
seed = 42
data = SyntheticRegressionData(w=tf.constant([2, -3.4, 5.]), b=4.2)
trainer = Trainer(max_epochs=3)

model = LinearRegression(dimensionality=dim)

design = f3dasm.make_nd_continuous_design(bounds=np.tile([-1.,1.], (dim+1, 1)), dimensionality=dim+1)

sampler = f3dasm.sampling.LatinHypercube(design=design,seed=seed)
N = sampler.get_samples(1)
w = N.get_input_data().to_numpy().ravel()

model.set_model_weights(w)


trainer.fit(model=model, data=data, optimizer=tf.keras.optimizers.SGD(0.03))

model2 = trainer.extract_model()
model2.get_model_weights()

array([[ 2.0008564],
       [-3.3875527],
       [ 4.987299 ],
       [ 4.187114 ]], dtype=float32)

In [16]:
for batch in data.train_dataloader():
    loss, gradient = trainer.evaluate(*batch[:-1],batch[-1])

    gr = get_flat_array_from_list_of_arrays(gradient)
    print(f"loss: {loss} \n\n gradient: {gr} \n\n gradient_orig: {gradient}")
    break

print(model.get_weights())

loss: 0.0006688890280202031 

 gradient: [[-0.0008425 ]
 [ 0.02829554]
 [-0.02690743]
 [-0.01597123]] 

 gradient_orig: [<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-0.0008425 ],
       [ 0.02829554],
       [-0.02690743]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.01597123], dtype=float32)>]
[array([[ 2.0008564],
       [-3.3875527],
       [ 4.987299 ]], dtype=float32), array([4.187114], dtype=float32)]


In [17]:
model.get_weights() - (0.03 * gr)

array([[array([[ 2.0008817],
               [-3.3875275],
               [ 4.987324 ]], dtype=float32),
        array([4.187139], dtype=float32)],
       [array([[ 2.0000076],
               [-3.3884015],
               [ 4.98645  ]], dtype=float32),
        array([4.186265], dtype=float32)],
       [array([[ 2.0016637],
               [-3.3867455],
               [ 4.9881063]], dtype=float32),
        array([4.187921], dtype=float32)],
       [array([[ 2.0013356],
               [-3.3870735],
               [ 4.987778 ]], dtype=float32),
        array([4.187593], dtype=float32)]], dtype=object)

In [18]:
get_reshaped_array_from_list_of_arrays(gr, gradient)

[array([[-0.0008425 ],
        [ 0.02829554],
        [-0.02690743]], dtype=float32),
 array([[-0.01597123]], dtype=float32)]

In [19]:
trainer.extract_model().weights

[<tf.Variable 'dense/kernel:0' shape=(3, 1) dtype=float32, numpy=
 array([[ 2.0008564],
        [-3.3875527],
        [ 4.987299 ]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([4.187114], dtype=float32)>]

In [20]:
model = trainer.extract_model()

In [21]:
model.get_weights()

[array([[ 2.0008564],
        [-3.3875527],
        [ 4.987299 ]], dtype=float32),
 array([4.187114], dtype=float32)]

In [22]:
design = f3dasm.make_nd_continuous_design(bounds=np.tile([-1.,1.], (dim+1, 1)), dimensionality=dim+1)

sampler = f3dasm.sampling.LatinHypercube(design=design,seed=seed)
N = sampler.get_samples(1)
w = N.get_input_data().to_numpy().ravel()
w

array([-0.25091976,  0.90142861,  0.46398788,  0.19731697])

In [23]:
s = [a.shape for a in model.model.get_weights()]

In [24]:
weights = get_reshaped_array_from_list_of_arrays(w, model.model.get_weights())

In [25]:
weights.

SyntaxError: invalid syntax (3307102578.py, line 1)

In [ ]:
model.model.get_weights().shape